In [2]:
from qiskit import *
from qiskit_aer import *
import numpy as np
import csv

# Function to create a random string of bits
def random_bit_string(length):
    return ''.join(np.random.choice(['0', '1'], size=length))

# Function to encode the qubits by Alice
def alice_encode(qc, bits, bases):
    for i in range(len(bits)):
        if bases[i] == 0:  # Prepare qubit in Z-basis
            if bits[i] == '1':
                qc.x(i)
        elif bases[i] == 1:  # Prepare qubit in X-basis
            if bits[i] == '0':
                qc.h(i)
            else:
                qc.x(i)
                qc.h(i)

# Function to measure qubits by Bob
def bob_measure(qc, bases):
    for i in range(len(bases)):
        if bases[i] == 0:  # Measure qubit in Z-basis
            qc.measure(i, i)
        elif bases[i] == 1:  # Measure qubit in X-basis
            qc.h(i)
            qc.measure(i, i)
            qc.h(i)

# Function to simulate Eve's attack
def eve_attack(qc):
    for i in range(n):
        qc.measure(i, i)

# Function to compare bases between Alice and Bob
def compare_bases(bases_a, bases_b):
    return [bases_a[i] == bases_b[i] for i in range(len(bases_a))]

# Function to extract the key bits agreed upon by Alice and Bob
def extract_key(bits, bases, agreed_bases):
    return ''.join([bits[i] for i in range(len(bits)) if agreed_bases[i]])

# Function to write Alice, Bob, and Eve's messages to a CSV file
def write_to_csv(alice_message, bob_message, eve_message):
    with open('bb84_messages.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Alice Message', 'Bob Message', 'Eve Message'])
        for a, b, e in zip(alice_message, bob_message, eve_message):
            writer.writerow([a, b, e])

# Parameters
n = 20  # Number of qubits
alice_bases = [np.random.randint(2) for _ in range(n)]  # Random bases for Alice
bob_bases = [np.random.randint(2) for _ in range(n)]  # Random bases for Bob

# Random bits for Alice
alice_bits = random_bit_string(n)
alice_message = [chr(int(alice_bits[i:i+8], 2)) for i in range(0, len(alice_bits), 8)]

# Quantum circuit setup
qc = QuantumCircuit(n, n)

# Alice encodes her bits
alice_encode(qc, alice_bits, alice_bases)

# Eve intercepts and measures the qubits
eve_attack(qc)

# Bob measures the qubits
bob_measure(qc, bob_bases)

# Simulate the circuit
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(qc, shots=1, memory=True)  # Specify memory=True to get individual measurement outcomes
result = job.result()
counts = result.get_counts()

# Extract Bob's bits from measurement outcomes
bob_bits_measured = list(counts.keys())[0]  # Bob's bits as a binary string

# Convert Bob's bits to bytes
bob_message = [bob_bits_measured[i:i+8] for i in range(0, len(bob_bits_measured), 8)]

# Convert Alice's bits to bytes
alice_bits_binary = ''.join([alice_bits[i] for i in range(len(alice_bits)) if alice_bases[i] == 0])

# Compare bases and extract the final key
agreed_bases = compare_bases(alice_bases, bob_bases)
final_key = extract_key(alice_bits_binary, alice_bases, agreed_bases)

# Write Alice, Bob, and Eve's messages to a CSV file
write_to_csv(alice_message, bob_message, alice_message)

# Print the results
print("Alice's message:", ''.join(alice_message))
print("Bob's message:", ''.join([chr(int(byte, 2)) for byte in bob_message]))
print("Final Key:", final_key)


Alice's message: ´H 
Bob's message: ¡c
Final Key: 1000000


In [25]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer

# Simulate a quantum hash function
def quantum_hash(input_string):
    binary_input = ''.join(format(ord(char), '08b') for char in input_string)
    length = len(binary_input)
    
    qc = QuantumCircuit(length, length)
    
    # Apply X gates where the binary input is 1
    for i in range(length):
        if binary_input[i] == '1':
            qc.x(i)
    
    # Apply Hadamard gates and CX gates for hashing
    qc.h(range(length))
    for i in range(0, length, 2):
        qc.cx(i, (i + 1) % length)
        qc.h(i)
    
    # Measure the qubits
    qc.measure(range(length), range(length))
    
    # Simulate the quantum circuit
    simulator = Aer.get_backend('qasm_simulator')
    result = simulator.run(qc,  shots=1024).result()
    counts = result.get_counts()
    
    # Get the most frequent result to ensure stability in the hash output
    hash_binary = max(counts, key=counts.get)
    return hash_binary

# BB84: Alice prepares and sends qubits
def bb84_protocol(n_bits=5, eve_involved=False):
    # Alice's random bits (message) and bases (0 = rectilinear, 1 = diagonal)
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)
    
    # Bob's random bases
    bob_bases = np.random.randint(2, size=n_bits)
    
    # Eve's random bases (if Eve is eavesdropping)
    eve_bases = np.random.randint(2, size=n_bits) if eve_involved else None
    
    # Alice's qubits sent to Bob
    sent_qubits = []
    
    for i in range(n_bits):
        qc = QuantumCircuit(1, 1)
        if alice_bases[i] == 1:
            qc.h(0)  # Diagonal basis
        
        if alice_bits[i] == 1:
            qc.x(0)  # Prepare bit 1
        
        # If Eve is eavesdropping, she measures the qubit
        if eve_involved:
            if eve_bases[i] == 1:
                qc.h(0)  # Eve measures in the diagonal basis
            qc.measure(0, 0)
            result = Aer.get_backend('qasm_simulator').run(qc,shots=1).result()
            counts = result.get_counts()
            eve_result = int(max(counts, key=counts.get))
            # Eve may collapse the qubit, so reset for Bob
            qc = QuantumCircuit(1, 1)
            if eve_result == 1:
                qc.x(0)
        
        sent_qubits.append(qc)

    # Bob measures the qubits
    bob_results = []
    for i in range(n_bits):
        qc = sent_qubits[i]
        if bob_bases[i] == 1:
            qc.h(0)  # Bob measures in the diagonal basis
        qc.measure(0, 0)
        
        result = Aer.get_backend('qasm_simulator').run(qc, shots=1).result()
        counts = result.get_counts()
        bob_results.append(int(max(counts, key=counts.get)))
    
    # Compare Alice's and Bob's bases
    matching_indices = np.where(alice_bases == bob_bases)[0]
    alice_key = alice_bits[matching_indices]
    bob_key = np.array(bob_results)[matching_indices]
    
    print(f"Alice's key: {alice_key}")
    print(f"Bob's key: {bob_key}")
    
    # Detect if there are mismatches in the keys (due to eavesdropping)
    if np.array_equal(alice_key, bob_key):
        print("No eavesdropping detected.")
    else:
        print("Warning: Eavesdropping detected!")
    
    return alice_key, bob_key

# Simulate Alice sending a message with BB84
def alice_to_bob_bb84(eve_involved=False):
    # Step 1: Run BB84 to establish a shared key
    alice_key, bob_key = bb84_protocol(n_bits=5, eve_involved=eve_involved)
    
    # Step 2: Alice sends a message (binary string)
    message = "01101"  # Example message in binary form
    print(f"Alice's original message: {message}")
    
    # Step 3: Alice encrypts the message using the shared key (XOR encryption)
    def xor_encrypt(message, key):
        encrypted = ''.join(str(int(m) ^ int(k)) for m, k in zip(message, key[:len(message)]))
        return encrypted
    
    encrypted_message = xor_encrypt(message, alice_key)
    print(f"Encrypted message: {encrypted_message}")
    
    # Step 4: Alice generates a quantum hash of the original message
    hash_value = quantum_hash(message)
    print(f"Quantum hash of the original message: {hash_value}")
    
    # Step 5: Bob decrypts the message using his key
    decrypted_message = xor_encrypt(encrypted_message, bob_key)
    print(f"Bob's decrypted message: {decrypted_message}")
    
    # If the message decrypted correctly, no eavesdropping; otherwise, Eve was present
    if message == decrypted_message:
        print("Message integrity verified: No tampering.")
    else:
        print("Warning: Message tampered due to eavesdropping!")

# Simulate BB84 without Eve
print("BB84 Simulation without Eve:")
alice_to_bob_bb84(eve_involved=False)

print("\nBB84 Simulation with Eve eavesdropping:")
alice_to_bob_bb84(eve_involved=True)


BB84 Simulation without Eve:
Alice's key: [1]
Bob's key: [1]
No eavesdropping detected.
Alice's original message: 01101
Encrypted message: 1
Quantum hash of the original message: 1100101000101000011010100110000000000010
Bob's decrypted message: 0

BB84 Simulation with Eve eavesdropping:
Alice's key: [1 0]
Bob's key: [0 1]
Alice's original message: 01101
Encrypted message: 11
Quantum hash of the original message: 0100000010100000111010101110101000100000
Bob's decrypted message: 10


In [46]:
import numpy as np

# Function to convert string to binary
def string_to_binary(message):
    return ''.join(format(ord(char), '08b') for char in message)

# Function to convert binary to string
def binary_to_string(binary):
    chars = [chr(int(binary[i:i + 8], 2)) for i in range(0, len(binary), 8)]
    return ''.join(chars)

# Function to perform XOR encryption
def xor_encrypt(message, key):
    return ''.join(str(int(m) ^ int(k)) for m, k in zip(message, key))

# Function for key distribution using BB84
def key_distribution(num_bits):
    alice_key = np.random.randint(0, 2, num_bits)
    bob_key = np.random.randint(0, 2, num_bits)
    
    # Check for eavesdropping
    if np.random.rand() < 0.5:  # Simulate a 50% chance of eavesdropping
        print("Warning: Eavesdropping detected!")
        alice_key = (alice_key + np.random.randint(0, 2, num_bits)) % 2  # Modify Alice's key
    else:
        print("No eavesdropping detected.")

    return alice_key, bob_key

# Function to send a message from Alice to Bob
def alice_to_bob(user_message):
    binary_message = string_to_binary(user_message)  # Convert message to binary
    num_bits = len(binary_message)  # Total bits needed for the message
    print(f"Alice's original message: {user_message}")
    
    # Step 1: Key Distribution
    alice_key, bob_key = key_distribution(num_bits)
    print(f"Alice's key: {alice_key}")
    print(f"Bob's key: {bob_key}")
    
    # Step 2: Encrypt the message
    encrypted_message = xor_encrypt(binary_message, alice_key)
    print(f"Encrypted message: {encrypted_message}")

    # Step 3: Bob decrypts the message
    decrypted_binary_message = xor_encrypt(encrypted_message, bob_key)
    decrypted_message = binary_to_string(decrypted_binary_message)  # Convert binary back to string
    print(f"Bob's decrypted message: {decrypted_message}")

# Main function
def main():
    user_message = input("Enter a message for Alice to send to Bob: ")
    
    # Simulate BB84
    alice_to_bob(user_message)

# Run the main function
main()


Alice's original message: rup
Alice's key: [0 0 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 1 1 1]
Bob's key: [1 0 1 1 1 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0]
Encrypted message: 011110001110011001111111
Bob's decrypted message: ÃVó
